# Car Detection using Transfer Learning
TensorFlow object detector pre-trained on the COCO dataset. Architecture: Inception SSD

Author: ottermegazord
Environment: Google Colab

# Setup

### Install dependencies and packages

In [0]:
!pip install -U --pre tensorflow=="2.*"
!pip install pycocotools

Download `tensorflow/models` into path or `cd` to parent directory of the repository if exists.

In [0]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Compile protobufs

In [0]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

Install the object_detection package

In [0]:
%%bash 
cd models/research
pip install .

### Imports

In [0]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display
import cv2
from google.colab.patches import cv2_imshow

VIDEO_STREAM = "sample_data/test_video.mp4"
VIDEO_STREAM_OUT = "sample_data/output_video.mp4"

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Import the object detection modules.

In [0]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

Patches:

In [0]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

# Model preparation 

In [0]:
def load_model(model_name):
  """
  Load model from TensorFlow repository
  params: String containing name of model
  """
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))
  model = model.signatures['serving_default']

  return model

In [0]:
PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

##### Test model on images

In [0]:
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('models/research/object_detection/test_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

In [0]:
model_name = 'ssd_inception_v2_coco_2018_01_28'
model = load_model(model_name)

##### Check model inputs and outputs

In [0]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  output_dict = model(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

### Inference

In [0]:
import cv2
def show_inference_video(model, video_path, video_stream_out):
  """
  Wrapper function that outputs image with bounding boxes
  params model: Model from TF object detection model
  params image_path: Path to test image
  """

  cap = cv2.VideoCapture(video_path)
  fourcc = cv2.VideoWriter_fourcc(*'MJPG')
  out = cv2.VideoWriter(video_stream_out, fourcc, 20.0, (1920,1080))
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.

  while (cap.isOpened()):
    ret, image_np = cap.read()
    # Actual detection.
    # fshape = image_np.shape
    # fheight = fshape[0]
    # fwidth = fshape[1]
    # print (fwidth)
    # print (fheight)
    output_dict = run_inference_for_single_image(model, image_np)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks_reframed', None),
        use_normalized_coordinates=True,
        line_thickness=8)
    
    out.write(image_np)
  out.release()
video_stream_out = 'PATH TO STREAM OUT'
video_path = 'PATH TO STREAM IN'
show_inference_video(model, video_path,video_stream_out)


In [0]:
from google.colab import drive
drive.mount('/content/drive')